---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('../data/spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [3]:
def answer_one():
    
    ratio = spam_data.target[spam_data.target == 1].count() / spam_data.target.count()
    
    return ratio *100



In [4]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from operator import itemgetter

def answer_two():
    vocab = CountVectorizer().fit(X_train)
   
    return max(vocab.get_feature_names(), key=lambda x: len(x))#Your answer here


In [6]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    vocab = CountVectorizer().fit(X_train)
    transformed_data = vocab.transform(X_train)
    classifier = MultinomialNB(alpha=0.1).fit(transformed_data,y_train)
    y_predicted = classifier.predict(vocab.transform(X_test))
    return roc_auc_score(y_test,y_predicted)


In [8]:
answer_three()

0.9720812182741116

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from operator import itemgetter

def answer_four(): 
    vocab = TfidfVectorizer().fit(X_train)
    feature_names = np.array(vocab.get_feature_names())

    X_train_Vectorized = vocab.transform(X_train)

    max_value_over_all_rows = X_train_Vectorized.max(0).toarray()[0]
    max_value_over_all_rows_sorted_index = max_value_over_all_rows.argsort()

    # small 20 words will be from first 20 index in max_value_over_all_rows_sorted_index
    words_sm = feature_names[max_value_over_all_rows_sorted_index[:20]]
    words_lg = feature_names[max_value_over_all_rows_sorted_index[-20:]]

    sm_tuples = [(x,y) for x,y in zip(words_sm,max_value_over_all_rows[max_value_over_all_rows_sorted_index[:20]])]
    lg_tuples = [(x,y) for x,y in zip(words_lg,max_value_over_all_rows[max_value_over_all_rows_sorted_index[-20:]])]
    
    sm_sorted = sorted(sm_tuples, key =lambda x: (x[1],x[0]))
    lg_sorted = sorted(lg_tuples, key =lambda x: (-x[1],x[0]))
    
    sm_series = pd.Series([x[1] for x in sm_sorted], index= [x[0] for x in sm_sorted])
    lg_series = pd.Series([x[1] for x in lg_sorted], index= [x[0] for x in lg_sorted])
    
    return (sm_series,lg_series)

In [10]:
answer_four()

(aaniye          0.074475
 athletic        0.074475
 chef            0.074475
 companion       0.074475
 courageous      0.074475
 dependable      0.074475
 determined      0.074475
 exterminator    0.074475
 healer          0.074475
 listener        0.074475
 organizer       0.074475
 pest            0.074475
 psychiatrist    0.074475
 psychologist    0.074475
 pudunga         0.074475
 stylist         0.074475
 sympathetic     0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 dtype: float64, 146tf150p    1.000000
 645          1.000000
 anything     1.000000
 anytime      1.000000
 beerage      1.000000
 done         1.000000
 er           1.000000
 havent       1.000000
 home         1.000000
 lei          1.000000
 nite         1.000000
 ok           1.000000
 okie         1.000000
 thank        1.000000
 thanx        1.000000
 too          1.000000
 where        1.000000
 yup          1.000000
 tick         0.980166
 blank        0.932702
 dty

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk 

def answer_five():
    vocab = TfidfVectorizer(min_df=3).fit(X_train)
    transformed_data = vocab.transform(X_train)
    classifier = MultinomialNB(alpha=0.1).fit(transformed_data,y_train)
    y_predicted = classifier.predict(vocab.transform(X_test))
    return roc_auc_score(y_test,y_predicted)

In [12]:
answer_five()

0.9416243654822335

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [13]:
def answer_six():
    not_spam_length = spam_data[spam_data.target==0].text.str.len().sum()/spam_data[spam_data.target==0].text.count()
    spam_length = spam_data[spam_data.target==1].text.str.len().sum()/spam_data[spam_data.target==1].text.count()
    
    return (not_spam_length, spam_length)

In [14]:
answer_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [15]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [16]:
from sklearn.svm import SVC

def answer_seven():
    vocab = TfidfVectorizer(min_df=5).fit(X_train)
    transformed_data = vocab.transform(X_train)
    X_train_new = add_feature(transformed_data,X_train.str.len() )
    classifier = SVC(C=10000).fit(X_train_new,y_train)
    X_test_new = add_feature(vocab.transform(X_test), X_test.str.len())
    y_predicted = classifier.predict(X_test_new)
    return roc_auc_score(y_test,y_predicted)

In [17]:
answer_seven()

0.9581366823421557

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [18]:
def answer_eight():
    
    not_spam_length = spam_data[spam_data.target==0].text.str.findall(r'\d').str.len().sum()/spam_data[spam_data.target==0].text.count()
    spam_length     = spam_data[spam_data.target==1].text.str.findall(r'\d').str.len().sum()/spam_data[spam_data.target==1].text.count()

    return (not_spam_length, spam_length)


In [19]:
answer_eight()

(0.2992746113989637, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [20]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    vocab = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train)
    transformed_data = vocab.transform(X_train)
    X_train_new = add_feature(transformed_data,[X_train.str.len(), X_train.str.findall(r'\d').str.len() ])
    classifier = LogisticRegression(C=100).fit(X_train_new,y_train)
    X_test_new = add_feature(vocab.transform(X_test), [X_test.str.len(), X_test.str.findall(r'\d').str.len()] )
    y_predicted = classifier.predict(X_test_new)
    return roc_auc_score(y_test,y_predicted)

In [21]:
answer_nine()

0.9653328353394565

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [22]:
def answer_ten():
    
    not_spam_length = spam_data[spam_data.target==0].text.str.findall(r'\W').str.len().sum()/spam_data[spam_data.target==0].text.count()
    spam_length     = spam_data[spam_data.target==1].text.str.findall(r'\W').str.len().sum()/spam_data[spam_data.target==1].text.count()

    return (not_spam_length, spam_length)

In [23]:
answer_ten()

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [24]:
def answer_eleven():
    
    vocab = CountVectorizer(analyzer='char_wb', min_df=5, ngram_range=(2,5)).fit(X_train)
    transformed_data = vocab.transform(X_train)
    X_train_new = add_feature(transformed_data,[X_train.str.len(), X_train.str.findall(r'\d').str.len(),X_train.str.findall(r'\W').str.len() ])
    classifier = LogisticRegression(C=100).fit(X_train_new,y_train)
    X_test_new = add_feature(vocab.transform(X_test), [X_test.str.len(), X_test.str.findall(r'\d').str.len(), X_test.str.findall(r'\W').str.len()] )
    y_predicted = classifier.predict(X_test_new)
    auc_score = roc_auc_score(y_test,y_predicted)

    features = vocab.get_feature_names()
    features+=['length_of_doc', 'digit_count', 'non_word_char_count']
    features_array=np.array(features)
    coeff_index_sorted = classifier.coef_[0].argsort()

    sm = features_array[coeff_index_sorted[:10]]
    lg = features_array[coeff_index_sorted[-1:-11:-1]]

    return (auc_score,list(sm),list(lg))



In [25]:
answer_eleven()

(0.9788593110707434,
 ['. ', '..', '? ', ' i', ' y', ' go', ':)', ' h', 'go', ' m'],
 ['digit_count', 'ne', 'ia', 'co', 'xt', ' ch', 'mob', ' x', 'ww', 'ar'])

In [26]:
import datetime
print("Last run at - "+ datetime.datetime.now().strftime('%d-%b-%Y %H:%M'))

Last run at - 25-May-2018 00:22
